In [15]:
using Revise
include("EFITModule/EFIT.jl")
include("EFITModule/ARBSCATT.jl")


Main.ARBSCATT

In [16]:
matGrid, params = Main.ARBSCATT.loadARBSCATT("TestFiles");

In [20]:
materials = [Main.EFIT.IsoMats["lightweightGeneric"],Main.EFIT.IsoMats["steel"]]

c = 8000
#Frequency, hz
f0=1e6
#Period
t0 = 1.00 / f0

dx = c/(8*f0)
dt = dx/(c*sqrt(3))


grid = Main.EFIT.EFITGrid(matGrid.+1,materials,dt,dx);

Creating grid of size 867, 716, 1431


In [21]:
function source(t)
    v = exp(-((2*(t-2*t0)/(t0))^2))*sin(2*pi*f0*t)*0.1
    return Data.Number(v)
end

source (generic function with 1 method)

In [22]:
sx = 715
sy = 358
sz = 866
r = 45

rx = sx-r:sx+r
ry = sy-r:sy+r
rz = sz:sz

nSteps = 500
for i = 1:nSteps
    if i % 10 == 0
        println("$i of $nSteps")


    end
    Main.EFIT.SimStep!(grid)
    @parallel (rx,ry,rz) Main.EFIT.applySource!(grid.vx,grid.vy,grid.vz, Data.Number(0.0), Data.Number(0.0), (source(t)))
end

10 of 500
20 of 500
30 of 500
40 of 500
50 of 500
60 of 500
70 of 500
80 of 500
90 of 500
100 of 500
110 of 500
120 of 500
130 of 500
140 of 500
150 of 500
160 of 500
170 of 500
180 of 500
190 of 500
200 of 500
210 of 500
220 of 500
230 of 500
240 of 500
250 of 500
260 of 500
270 of 500
280 of 500
290 of 500
300 of 500
310 of 500
320 of 500
330 of 500
340 of 500
350 of 500
360 of 500
370 of 500
380 of 500
390 of 500
400 of 500
410 of 500
420 of 500
430 of 500
440 of 500
450 of 500
460 of 500
470 of 500
480 of 500
490 of 500
500 of 500
